In [1]:
!pip install pandas
!pip install numpy
!pip install plotly
!pip install scikit-learn
!pip install findspark
!pip install pyspark

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/11/17/fb1a34f3e73debbc2fd15a01ea17eaab3717943d08463ff4979a4f024b3f/pandas-2.1.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for numpy<2,>=1.23.2 from https://files.pythonhosted.org/packages/99/2b/f7114983d84303019385d93d24d729aedba67be7e083286f114188943cf3/numpy-1.26.3-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-1.26.3-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.1 from https://files.pythonhosted.org/packages/a3/fb/52b62131e21b24ee297e4e95ed41eba29647dad0e0051a92bb66b43c70ff/tzdata-2023.4-py2.py3-none-any.whl.metadata
Using cached pandas-2.1.4-cp311-cp311-win_amd64.whl (10.6 MB)
Using cached numpy-1.26


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
Using cached plotly-5.18.0-py3-none-any.whl (15.6 MB)
Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/4e/ba/ce9bd1cd4953336a0e213b29cb80bb11816f2a93de8c99f88ef0b446ad0c/scikit_learn-1.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/43/d0/f3cd75b62e1b90f48dbf091261b2fc7ceec14a700e308c50f6a69c83d337/scipy-1.11.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.4 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 643.1 kB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.wh


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pyspark-3.5.0.tar.gz (316.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425385 sha256=65616105358c2d866f4db5a6658a7219f2dff521dd23c5e9c33ce22d8636ccab
  Stored in directory: c:\users\korog\appdata\local\pip\cache\wheels\38\df\61\8c121f50c3cffd77f8178180dd232d90b3b99d1bd61fb6d6be
Successfully built pyspark



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import pyspark
import findspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
import socket
import platform

In [3]:
hostname = socket.gethostname()
IP = socket.gethostbyname(hostname)

system_info = platform.uname()
 
print("Computer IP Address:", IP)
print("Computer Name:", hostname)
print("\nSystem Information:")
print("System:", system_info.system)
print("Node (Machine) Name:", system_info.node)
print("Release:", system_info.release)
print("Version:", system_info.version)
print("Machine (Processor):", system_info.machine)
print("Processor (CPU):", system_info.processor)

findspark.init()

conf = pyspark.SparkConf()
spark= SparkSession.builder.config(conf=conf).getOrCreate()

df = spark.read.csv('users_and_ratings.csv', header=True, inferSchema=True)
genre_columns = df.columns[2:-2]
assembler = VectorAssembler(inputCols=genre_columns, outputCol="features")
df = assembler.transform(df)

Computer IP Address: 192.168.1.194
Computer Name: Emre

System Information:
System: Windows
Node (Machine) Name: Emre
Release: 10
Version: 10.0.22631
Machine (Processor): AMD64
Processor (CPU): Intel64 Family 6 Model 158 Stepping 10, GenuineIntel


In [4]:
train_df, test_df = df.randomSplit([0.7, 0.3], seed=5024)

def als_algo(rank, iter, lm):

    als = ALS(
        rank=rank,
        maxIter=iter,
        regParam=lm,
        userCol="user_id",
        itemCol="anime_id",
        ratingCol="rating",
        coldStartStrategy="drop",
        seed=5024
    )

    model = als.fit(train_df)

    rmse_evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    mse_evaluator = RegressionEvaluator(metricName="mse", labelCol="rating", predictionCol="prediction")
    
    predictions = model.transform(test_df)
    user_recommendations = model.recommendForAllUsers(10)
    
    rmse = rmse_evaluator.evaluate(predictions)
    mse = mse_evaluator.evaluate(predictions)
    
    print(f"For rank: {rank} / iteration: {iter} / lambda: {lm}")
    print(f"Root Mean Squared Error (RMSE) = {rmse}\nMean Squarred Error (MSE) = {mse}")
    
    predictions.select("user_id", "name", "rating", "prediction").show(10)
    
    user_recommendations.show(10,truncate=False)

als_algo(47,10,0.01)

For rank: 47 / iteration: 10 / lambda: 0.01
Root Mean Squared Error (RMSE) = 1.3465874306079695
Mean Squarred Error (MSE) = 1.8132977082713728
+-------+--------------------+------+----------+
|user_id|                name|rating|prediction|
+-------+--------------------+------+----------+
|    183|Densetsu no Yuush...|     4|  6.321925|
|    384|One Piece: Mamore...|     6| 5.1455455|
|    385|Magi: Sinbad no B...|     9|  9.011071|
|    392|Break Blade 4: Sa...|     6|  7.863145|
|    392|Magi: Sinbad no B...|     8|  6.678406|
|    392|Densetsu no Yuush...|     9|  7.305724|
|    460|Zoku Natsume Yuuj...|     8|  7.450839|
|    497|Mobile Suit Gunda...|     8|  9.030138|
|    847|Ansatsu Kyoushits...|     9|  8.494466|
|    847|    Magic Kaito 1412|     9| 8.0196495|
+-------+--------------------+------+----------+
only showing top 10 rows

+-------+----------------------------------------------------------------------------------------------------------------------------------------